In [ ]:
import os
# import requests
# import streamlit as st
from dotenv import load_dotenv

import google.generativeai as genai
# from google.generativeai import GenerationConfig
from langchain_google_genai import ChatGoogleGenerativeAI

: 

In [ ]:
load_dotenv()

In [ ]:
api_key = os.getenv("GOOGLE_API_KEY") # GEMINI_API_KEY 키에 해당하는 value를 가져온다 (=api key)
print(api_key[:-5]) # 보안상의 이유로 끝의 5자리는 생략!

In [ ]:
# api key를 등록, 저장
# 이 코드를 실행함으로써 Gemini 모델 객체를 만들거나 generate_content()와 같은 api 호출 메소드를 반복해서 실행할 때 매번 key를 넘기지 않아도 된다. -> 일종의 프로젝트 key 전역설정
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Gemini API 가격 및 사용 가능 토큰 확인
https://ai.google.dev/gemini-api/docs/pricing?hl=ko

In [ ]:
model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
model

# Chapter 1. Gemini에 직접 질문을 해서 답변을 받아보자!

## reponse : LLM 모델의 답변
- response.text → 최종 생성 텍스트
- response.candidates → 여러 후보 응답
- response.usage_metadata → 토큰 사용량 정보

etc...

In [ ]:
# 1. generate_content(text) 내의 문장(text)을 프롬프트로 LLM 모델에게 전달
# 2. generate_content() : Google Gemini API 서버에 요청
# 3. response : LLM 모델이 내놓은 답변 (여러 정보 확인 가능)
response = model.generate_content("안녕하세요, Gemini API 테스트 중입니다.") # 질문 입력

In [ ]:
response

In [ ]:
print(response.text)

# Chapter 2. 우리의 대화를 기억하게 하자
- 지금까지의 대화 세션을 저장, 메모리 캐싱해서 추가적인 질문 없이 Gemini가 직접 대화를 검색하도록 하는 방법
> 동일한 질문에 대한 답변을 저장, 메모리에 저장된 답변을 반환
- 토큰 사용량이 줄어듦
- 대답 속도 향상

In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

In [ ]:
%%time

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

prompt = PromptTemplate.from_template("{country}에 대해서 설명해줘. 최대한 짧게!")

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"country": "미국"}))

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response)

# Chapter 3. 우리가 원하는 포맷으로 답변을 받아보자!
- Few Shot 프롬프트 : 원하는 답변 포맷을 여러 예시로 제공하는 방법
- LLM 모델은 Few shot에서 지정한 포맷을 맞춰 답변을 제공

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

In [ ]:
examples = [
    {
        "question": "최근 가장 급등한 주식 종목 알려줘",

        "answer": """
추가 질문: 미국 주식 중 3개를 알려줘
중간 답변: 최근 엔비디아, 구글, 메타가 가장 많이 올랐습니다.
최종 답변은: 엔비디아, 테슬라, 팔란티어가 가장 많이 올랐습니다.
"""}
]

In [ ]:
example_prompt = PromptTemplate.from_template(
"Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [ ]:
question = "최근 1년 동안 가장 많이 오른 한국 주식 알려줘"
final_prompt = prompt.format(question=question)
print(final_prompt)

In [ ]:
response = model.generate_content(final_prompt)

In [ ]:
print(response.text)